In [ ]:
# Setting up the Colab environment. DO NOT EDIT!
try:
  from applied_biostats import setup_environment
except ImportError:
  !pip -q install applied-biostats-helper
  from applied_biostats import setup_environment
finally:
  grader = setup_environment('Module06_walkthrough')

# Walkthrough

## Learning Objectives
At the end of this learning activity you will be able to:
* Create categorical comparisons with countplots.
* Create quantitative comparison plots with seaborn: stripplot, barplot, boxplot with Seaborn.
* Create correlation style plots with with scatterplot and regplot.
* Utilize `pd.melt` to plot wide data with seaborn.
* Describe bootstapping and confidence intervals.

This week we will continue our exploration of data from a cohort study participants of People Living with HIV (PLwH) here at Drexel.

As we discussed in the introduction, this data collection effort was done to provide a resource for many projects across the fields of HIV, aging, inflammation, neurocognitive impairment, immune function, and unknowable future projects.
In this walkthrough we will explore a collection of cytokines and chemokines measured by a Luminex panel of common biomarkers of inflammation.
We use this data to look for correlations between cytokine biomarkers and demographic variables.

### Documentation

I HIGHLY recommend exploring and utilizing the documentation for this tool:

* Base: https://seaborn.pydata.org/
* Gallery of great examples: https://seaborn.pydata.org/examples/index.html
* Function documentation: (We will be using the _Function Interface_. https://seaborn.pydata.org/api.html#function-interface
* Tutorial: https://seaborn.pydata.org/tutorial.html

When I'm making figures I'll usually have the documentation open on another tab.

---------------------------------------------

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# This is how we normally import seaborn
import seaborn as sns

%matplotlib inline

In [ ]:
data = pd.read_csv('cytokine_data.csv')
data.head()

## Categorical Comparisons

### Counting with `countplot`

In [ ]:
sns.countplot(data = data, # Pass the Dataframe `data` into the kwarg `data`
              x = 'neuro_screen_impairment_level')

In [ ]:
sns.countplot(data = data, # Pass the Dataframe `data` into the kwarg `data`
              x = 'neuro_screen_impairment_level',
              hue = 'cocaine_use')

If we wanted to look at multiple columns.
It would be wonderful if we could do this:

In [ ]:
# This doesn't work

# sns.countplot(data = data,
#               x = 'neuro_screen_impairment_level',
#               hue = ['cocaine_use', 'cannabinoid_use'])

But, we can create a new column that is the combination of the two:

In [ ]:
def use_desc(row):
    cocaine = 'Y' if row['cocaine_use'] else 'N'
    cannabinoid = 'Y' if row['cannabinoid_use'] else 'N'
    return cocaine+cannabinoid

data['multi_use'] = data.apply(use_desc, axis=1)

While we're doing transformations, let's improve our plots by converting our `neuro_screen_impairment_level` to a proper ordinal variable.
 - Categorical variables : A set of distinct and seperable groups.
 - Ordinal variables : A set of distinct and seperable groups with a sortable order.

By default, `pandas` treats all strings as `categorical` and it sorts them alphabetically.
But sometimes, we want to specify a order. We can do that like so.

In [ ]:
pd.Categorical(data['neuro_screen_impairment_level'],
               categories = ['none', 'mild', 'impaired'], ordered=True)

In [ ]:
data['neuro_screen_ordinal'] = pd.Categorical(data['neuro_screen_impairment_level'],
                                               categories = ['none', 'mild', 'impaired'], ordered=True)

In [ ]:
sns.countplot(data = data, # Pass the Dataframe `data` into the kwarg `data`
              x = 'neuro_screen_ordinal', # Seaborn will respect the ordering of our categories.
              hue = 'multi_use')

### Visualizing differences across categories with `stripplot`

In [ ]:
sns.stripplot(data = data,
              x = 'neuro_screen_ordinal',
              y = 'mcp1')

In [ ]:
sns.stripplot(data = data,
              x = 'neuro_screen_ordinal',
              hue = 'Sex',
              y = 'mcp1')
# Try dodge = True

You can even put plots on top of plots!

In [ ]:
# Grab the axis object that was created
ax = sns.stripplot(data = data,
                   x = 'neuro_screen_ordinal',
                   hue = 'Sex',
                   y = 'mcp1',
                   dodge = True)


sns.boxplot(data = data,
              x = 'neuro_screen_ordinal',
              hue = 'Sex',
              y = 'mcp1',
              ax = ax) # Give that axis to the next plot

## Quantifying the uncertainty of estimates

When presenting data it is important to rigorously convey the precision and uncertainty of our estimates.
I HIGHLY recommend reading the error bar tutorial in seaborn.
Below is a quick summary.


> The error bars around an estimate of central tendency can show one of two general things: either the range of uncertainty about the estimate or the spread of the underlying data around it.
> These measures are related: given the same sample size, estimates will be more uncertain when data has a broader spread. But uncertainty will decrease as sample sizes grow, whereas spread will not.


![Error bar taxonomy](https://seaborn.pydata.org/_images/error_bars_2_0.svg)

* Value Spread: How much each value differs from every other value
  * Standard deviation: Average difference of each point from the mean.
  * Percentile Interval: Middle range that contains some percent of the data.
* Estimate Uncertainty: What is the reasonable range of my estimate
  * Standard Error: Standard deviation/sqrt(sample_size)
  * Confidence Intervals:
  > The nonparametric approach to representing uncertainty uses bootstrapping: a procedure where the dataset is randomly resampled with replacement a number of times, and the estimate is recalculated from each resample. This procedure creates a distribution of statistics approximating the distribution of values that you could have gotten for your estimate if you had a different sample.

### Measuring Spread

Quantifies how much each individual measurement vary around the middle.

In [ ]:
# Parametric Assumptions

fig, (par_ax, np_ax) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

## Plot with parametric assumpitions
sns.barplot(data = data,
            x = 'neuro_screen_ordinal',
            y = 'mcp1',
            estimator = 'mean',
            errorbar = ('sd', 2),
                ax = par_ax) # 95% CI

sns.stripplot(data = data,
              x = 'neuro_screen_ordinal',
              y = 'mcp1',
              alpha=0.5,
              ax=par_ax)
par_ax.set_title('Parametric')

## Plot with non-parametric methods

## Plot with parametric assumpitions
sns.barplot(data = data,
            x = 'neuro_screen_ordinal',
            y = 'mcp1',
            estimator = 'median',
            errorbar = ('pi', 95),
                ax = np_ax) # 95% CI

sns.stripplot(data = data,
              x = 'neuro_screen_ordinal',
              y = 'mcp1',
              alpha=0.5,
              ax=np_ax)

np_ax.set_title('Non-Parametric')

fig.tight_layout()

### Measuring Uncertainty

Quantifies how confident we are about where the middle of the distribution is.

In [ ]:
# Parametric Assumptions

fig, (par_ax, np_ax) = plt.subplots(1, 2, figsize=(8, 4), sharey=True)

## Plot with parametric assumpitions
sns.barplot(data = data,
            x = 'neuro_screen_ordinal',
            y = 'mcp1',
            estimator = 'mean',
            errorbar = ('se', 2),
                ax = par_ax)

sns.stripplot(data = data,
              x = 'neuro_screen_ordinal',
              y = 'mcp1',
              alpha=0.25,
              ax=par_ax)
par_ax.set_title('Parametric')

## Plot with non-parametric methods
sns.barplot(data = data,
            x = 'neuro_screen_ordinal',
            y = 'mcp1',
            estimator = 'median',
            errorbar = ('ci', 95),
                ax = np_ax)

sns.stripplot(data = data,
              x = 'neuro_screen_ordinal',
              y = 'mcp1',
              alpha=0.25,
              ax=np_ax)

np_ax.set_title('Non-Parametric')

fig.tight_layout()

With hundreds of data points, we are very confident and precise about our estimate of the middle of the distribution.

It is important to understand the effect of sample size on spread and uncertainty.
Below is a simulation that picks small subsets of the data and generates the same plot.
Before you generate it; which do you think will change with sample-size and which will stay the same?

In [ ]:
def overlapped_plot(ax, df, y = 'mcp1', mode = 'spread'):
    "Create overlapped plot with different confidence assumptions"

    if mode == 'spread':
        # Use Standard Deviation
        eb = ('sd', 2)
    elif mode == 'uncertainty':
        # Use Standard error
        eb = ('se', 2)
    else:
        raise ValueError(f'Expected spread or uncertainty, got: {mode}')

    sns.barplot(data = df,
                x = 'neuro_screen_ordinal',
                y = y,
                errorbar = eb,
                ax = ax)

    sns.stripplot(data = df,
                  x = 'neuro_screen_ordinal',
                  y = y,
                  alpha=0.25,
                  ax=ax)


In [ ]:
NBINS = 10

BINS = np.linspace(10, len(data.index), NBINS)
fig, axs = plt.subplots(len(BINS), 2, figsize=(8, 50), sharex=True, sharey=True)

for row, sample_size in enumerate(BINS):
    print('Sampling', int(sample_size), 'samples for figure.')
    ndf = data.sample(n=int(sample_size))

    spread_ax, uncer_ax = axs[row,:]

    ## Plot with parametric assumpitions
    overlapped_plot(spread_ax, ndf, mode='spread')
    spread_ax.set_title(f'Spread: n = {sample_size:0.0f}')

    ## Plot with parametric assumpitions
    overlapped_plot(uncer_ax, ndf, mode='uncertainty')
    uncer_ax.set_title(f'Uncertainty: n = {sample_size:0.0f}')


In [ ]:
# Which is impacted by sample size?
# answer: 'spread', 'uncertainty', 'both', 'neither'
impact_of_sample_size = ...

In [ ]:
grader.check("impact_of_sample_size")

So, how are the two measures impacted by sample size?
  - Spread : Sample size has no effect on spread.
  - Uncertainty : Sample size reduces uncertainty.

This type of categorical visualization is limited in that it only considers a single summary statistic.
It ignores the "shape" of the data.
We can use histograms to explore this.

## Comparing Distributions

Seaborn makes simple graphs easy.

In [ ]:
sns.histplot(data = data,
             x = 'mcp1',
             hue = 'neuro_screen_ordinal')

If we don't like overlapping plots we can make it like a heatmap where the color indicates the count.

In [ ]:
sns.histplot(data = data,
             x = 'mcp1',
             y  = 'neuro_screen_ordinal',
             cbar = True)

Aggregate statistic to compute in each bin.
* count: show the number of observations in each bin
* frequency: show the number of observations divided by the bin width
* probability or proportion: normalize such that bar heights sum to 1
* percent: normalize such that bar heights sum to 100
* density: normalize such that the total area of the histogram equals 1

In [ ]:
sns.histplot(data = data,
             x = 'mcp1',
             hue = 'neuro_screen_ordinal',
             stat = 'percent',
             common_norm=False) # Each group should have its own "sum to 100"
# hue_order = ['none', 'mild']

## Measuring Correlation

The previous figures measured one (or more) categorical variables and a single quantitative variable.
If we want to understand how correlated any two quantitative variables are to each other.

In [ ]:
sns.scatterplot(data = data,
                x = 'mcp1', 
                y = 'tnfalpha')

We can also use a _regression plot_ to draw a best fit line with a confidence interval indicated by a shadow.

In [ ]:
sns.regplot(data = data,
            x = 'mcp1',
            y = 'tnfalpha')

## Figure Level Interface

All of the code we've been running above generates a single figure.
The functions below automate generating multiple figures across different facets of the data.
This is an incredibly useful way to explore relationships in your data.

### Categorical with `catplot`

In [ ]:
sns.catplot(data = data,
            x = 'neuro_screen_ordinal',
            y = 'mcp1',
            col = 'multi_use')

# Try:
# col_wrap = 2
# kind = 'bar', box, etc
# Other groupings across row and hue
#  Sex, isAA
#  pd.cut(data['Age'], [18, 30, 50, 100], labels=['young', 'adult', 'aged'])
#  pd.cut(data['tnfalpha'], 3, labels=['low', 'med', 'high'])

Each of the plots splits the data into a different categorical subset, then only plots the data within that set.
Use the examples in the comments to explore ways to visualize the data.

### Relational with `relplot`

In [ ]:
sns.relplot(data = data,
            x = 'tnfalpha',
            y = 'mcp1',
            col = 'multi_use')

# Try:
# col_wrap = 2
# kind = 'bar', box, etc
# Other groupings across row and hue
#  Sex, isAA
#  pd.cut(data['Age'], [18, 30, 50, 100], labels=['young', 'adult', 'aged'])
#  pd.cut(data['tnfalpha'], 3, labels=['low', 'med', 'high'])

### Linear model regression plots with `lmplot`

In [ ]:
sns.lmplot(data = data,
            x = 'tnfalpha',
            y = 'mcp1',
            col = 'multi_use')
# Try:
# col_wrap = 2
# kind = 'bar', box, etc
# Other groupings across row and hue
#  Sex, isAA
#  pd.cut(data['Age'], [18, 30, 50, 100], labels=['young', 'adult', 'aged'])
#  pd.cut(data['tnfalpha'], 3, labels=['low', 'med', 'high'])

## Plotting Multiple Columns

You'll probably notice that all of the graphs so far have visualized a single cytokine.
What if we wanted multiple cytokines in the same figure?

In [ ]:
# This doesn't work
# sns.stripplot(data = data,
#               x = 'neuro_screen_category',
#               hue = 'Sex',
#               y = ['mcp1', 'tnfalpha'],
#               dodge = True)

### I'm `pd.melt`ing

Seaborn only plots _long_ data.
We need to convert our _wide_ data into _long_ data first.
This is melting.

In [ ]:
data.head()

In [ ]:
melted_data = pd.melt(data,
                      id_vars = ['Sex', 'Age', 'neuro_screen_ordinal', 'multi_use'], # What we want replicated across all rows of the same sample
                      value_vars = ['mcp1', 'tnfalpha', 'il6'], # What columns we want to melt into rows
                      value_name = 'concentration', # The name of the value column after melting
                      var_name = 'cytokine') # The name of the variable column after melting
melted_data

In [ ]:
sns.catplot(data = melted_data,
            x = 'neuro_screen_ordinal',
            y = 'concentration',
            col = 'cytokine',
            hue = 'Sex',
            sharey=False,
            kind = 'bar')

Whether to use melting data and figure-level seaborn functions or using `plt.subplots` and then doing axis-level figure generation is a matter of personal perference and the situation.

## Submission

You do not need to submit this walkthrough notebook.
Simply complete the quiz.

---------------------------------------------